In [333]:
import pandas as pd
import os
import ast
import re

# Init - Creating a basic dataframe for further processing
- Here we import the dataset and create a 1x1 pandas dataframe
- Each row in the frame contains the unprocessed request/response pair. 


In [334]:
# Dataset filepath
imports_folder = "../data/imports"
dataset_filename = "flow-2_output.txt" # File to import
dataset_path = os.path.join(imports_folder, dataset_filename)

# Exported csv file:
exports_folder = "../data/exports"
exported_filename = "tmp.csv"
exported_csv_filepath = os.path.join(exports_folder, exported_filename)

In [335]:
# Matches the IP/port pattern that marks the start of a request.
request_start_pattern = r"^\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}(:\d+)?"

def parse_document_to_dataframe(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    requests = []
    current_request = []
    
    for line in lines:
        # Detect start of a new request.
        if re.match(request_start_pattern, line):
            # If there's an existing request being accumulated, add it to the requests list.
            if current_request:
                requests.append('\n'.join(current_request))
                current_request = []  # Reset for next request.

        # Accumulate lines for the current request.
        current_request.append(line)

    # Add the last request if it exists.
    if current_request:
        requests.append('\n'.join(current_request))

    # Create a DataFrame with each request as a row in a single column.
    df = pd.DataFrame(requests, columns=['Req.Start'])

    return df

initial_df = parse_document_to_dataframe(dataset_path)
initial_df

,Req.Start
0,172.20.10.11:57731: GET https://p31-quota.iclo...
1,172.20.10.11:57756: GET https://clients1.googl...
2,172.20.10.11:57756: GET https://clients1.googl...
3,172.20.10.11:57756: GET https://clients1.googl...
4,172.20.10.11:57756: GET https://clients1.googl...
5,172.20.10.11:57756: GET https://clients1.googl...
6,172.20.10.11:57756: GET https://clients1.googl...
7,172.20.10.11:57756: GET https://clients1.googl...
8,172.20.10.11:57756: GET https://clients1.googl...
9,172.20.10.11:57756: GET https://clients1.googl...


# Init - Splitting the column in two!
- Here we split request/response pair into two columns

In [336]:
def split_requests_and_responses(df):
    # Ensure 'Request' column is treated as a string
    df['Req.Start'] = df['Req.Start'].astype(str)

    # Split the string into two parts, then assign it to the columns 'Req.Start' and 'Res.Start'
    split_result = df['Req.Start'].str.split(' <<', n=1, expand=True)
    df['Req.Start'] = split_result[0]
    df['Res.Start'] = split_result[1].apply(lambda x: ' <<' + x if x is not None else '') # If no response set's it to None

    return df

df_req_res_pair = split_requests_and_responses(initial_df)
df_req_res_pair

,Req.Start,Res.Start
0,172.20.10.11:57731: GET https://p31-quota.iclo...,<< 200 532b\n\n Server: AppleHttpServer/7...
1,172.20.10.11:57756: GET https://clients1.googl...,"<< HTTP/2.0 200 OK 152b\n\n date: Sun, 25 ..."
2,172.20.10.11:57756: GET https://clients1.googl...,"<< HTTP/2.0 200 OK 148b\n\n date: Sun, 25 ..."
3,172.20.10.11:57756: GET https://clients1.googl...,"<< HTTP/2.0 200 OK 147b\n\n date: Sun, 25 ..."
4,172.20.10.11:57756: GET https://clients1.googl...,"<< HTTP/2.0 200 OK 137b\n\n date: Sun, 25 ..."
5,172.20.10.11:57756: GET https://clients1.googl...,"<< HTTP/2.0 200 OK 137b\n\n date: Sun, 25 ..."
6,172.20.10.11:57756: GET https://clients1.googl...,"<< HTTP/2.0 200 OK 166b\n\n date: Sun, 25 ..."
7,172.20.10.11:57756: GET https://clients1.googl...,"<< HTTP/2.0 200 OK 179b\n\n date: Sun, 25 ..."
8,172.20.10.11:57756: GET https://clients1.googl...,"<< HTTP/2.0 200 OK 126b\n\n date: Sun, 25 ..."
9,172.20.10.11:57756: GET https://clients1.googl...,"<< HTTP/2.0 200 OK 102b\n\n date: Sun, 25 ..."


# Further refinement of the request column
- Now we take the Request column and split it into further columns for easier processing. The new columns will be:
    - Req.Start
    - Req.Headers
    - Req.Content

In [337]:
def extract_headers_and_content(df):
    # Initialize new columns
    df['Req.Headers'] = None
    df['Req.Content'] = None

    # Iterate through each row in DataFrame
    for index, row in df.iterrows():
        # Split the request into potential headers and content
        parts = row['Req.Start'].split('\n\n\n\n', 1)

        request_part = parts[0]  # The first part will always be there
        content_part = parts[1] if len(parts) > 1 else None  # Content part, if it exists

        # First, separate the initial line from the headers
        initial_line, headers_part = request_part.split('\n\n', 1) if '\n\n' in request_part else (request_part, None)

        # Then Split headers into a list, if headers exist
        headers = []
        if headers_part:
            raw_headers = headers_part.split('\n\n')
            for header in raw_headers:
                trimmed_header = header.strip() # Trim whitespace
                headers.append(f'{trimmed_header}') # Append the header to the array

        # Update the DataFrame
        df.at[index, 'Req.Start'] = initial_line    
        df.at[index, 'Req.Headers'] = headers    
        df.at[index, 'Req.Content'] = content_part

    return df

df_req_res_pair = extract_headers_and_content(df_req_res_pair)
df_req_res_pair

,Req.Start,Res.Start,Req.Headers,Req.Content
0,172.20.10.11:57731: GET https://p31-quota.iclo...,<< 200 532b\n\n Server: AppleHttpServer/7...,"[Host: p31-quota.icloud.com:443, X-MMe-Timezon...",
1,172.20.10.11:57756: GET https://clients1.googl...,"<< HTTP/2.0 200 OK 152b\n\n date: Sun, 25 ...",[user-agent: Mozilla/5.0 (iPhone; CPU iPhone O...,json: t\n\n nolabels: t\n\n clie...
2,172.20.10.11:57756: GET https://clients1.googl...,"<< HTTP/2.0 200 OK 148b\n\n date: Sun, 25 ...",[user-agent: Mozilla/5.0 (iPhone; CPU iPhone O...,json: t\n\n nolabels: t\n\n clie...
3,172.20.10.11:57756: GET https://clients1.googl...,"<< HTTP/2.0 200 OK 147b\n\n date: Sun, 25 ...",[user-agent: Mozilla/5.0 (iPhone; CPU iPhone O...,json: t\n\n nolabels: t\n\n clie...
4,172.20.10.11:57756: GET https://clients1.googl...,"<< HTTP/2.0 200 OK 137b\n\n date: Sun, 25 ...",[user-agent: Mozilla/5.0 (iPhone; CPU iPhone O...,json: t\n\n nolabels: t\n\n clie...
5,172.20.10.11:57756: GET https://clients1.googl...,"<< HTTP/2.0 200 OK 137b\n\n date: Sun, 25 ...",[user-agent: Mozilla/5.0 (iPhone; CPU iPhone O...,json: t\n\n nolabels: t\n\n clie...
6,172.20.10.11:57756: GET https://clients1.googl...,"<< HTTP/2.0 200 OK 166b\n\n date: Sun, 25 ...",[user-agent: Mozilla/5.0 (iPhone; CPU iPhone O...,json: t\n\n nolabels: t\n\n clie...
7,172.20.10.11:57756: GET https://clients1.googl...,"<< HTTP/2.0 200 OK 179b\n\n date: Sun, 25 ...",[user-agent: Mozilla/5.0 (iPhone; CPU iPhone O...,json: t\n\n nolabels: t\n\n clie...
8,172.20.10.11:57756: GET https://clients1.googl...,"<< HTTP/2.0 200 OK 126b\n\n date: Sun, 25 ...",[user-agent: Mozilla/5.0 (iPhone; CPU iPhone O...,json: t\n\n nolabels: t\n\n clie...
9,172.20.10.11:57756: GET https://clients1.googl...,"<< HTTP/2.0 200 OK 102b\n\n date: Sun, 25 ...",[user-agent: Mozilla/5.0 (iPhone; CPU iPhone O...,json: t\n\n nolabels: t\n\n clie...


# Further Request processing part 2: Electric boogaloo
- The "Req.Start" column will now be processed to contain the IP, PORT, HTTP Method, URL and HTTP version (if it exists)

In [338]:
def parse_req_start(df):
    # Prepare lists to hold the parsed data
    methods = []
    source_ips = []
    source_ports = []
    urls = []
    http_versions = []

    # Regular expression pattern for parsing
    pattern = r"^(?P<ip>\d{1,3}(?:\.\d{1,3}){3}):(?P<port>\d+): (?P<method>\w+) (?P<url>https?://[^ ]+)(?: (?P<http_version>HTTP/\d\.\d))?"

    # Parse each row in the original DataFrame
    for row in df['Req.Start']:
        match = re.match(pattern, row)
        if match:
            methods.append(match.group('method'))
            source_ips.append(match.group('ip'))
            source_ports.append(match.group('port'))
            urls.append(match.group('url'))
            http_versions.append(match.group('http_version'))
        else:
            # Append None or appropriate defaults if the pattern does not match
            methods.append(None)
            source_ips.append(None)
            source_ports.append(None)
            urls.append(None)
            http_versions.append(None)

    # Add the parsed data as new columns to the original DataFrame
    df['Req.Method'] = methods
    df['Req.Source_IP'] = source_ips
    df['Req.Source_Port'] = source_ports
    df['Req.URL'] = urls
    df['Req.HTTP_Version'] = http_versions
    
    # Remove the Req.Start column as it's no longer needed
    original_df = df.drop('Req.Start', axis=1)

    return original_df

# Use the function to parse the original DataFrame and create a new one
df_req_res_pair = parse_req_start(df_req_res_pair)
df_req_res_pair 

,Res.Start,Req.Headers,Req.Content,Req.Method,Req.Source_IP,Req.Source_Port,Req.URL,Req.HTTP_Version
0,<< 200 532b\n\n Server: AppleHttpServer/7...,"[Host: p31-quota.icloud.com:443, X-MMe-Timezon...",,GET,172.20.10.11,57731,https://p31-quota.icloud.com/quotaservice/exte...,None
1,"<< HTTP/2.0 200 OK 152b\n\n date: Sun, 25 ...",[user-agent: Mozilla/5.0 (iPhone; CPU iPhone O...,json: t\n\n nolabels: t\n\n clie...,GET,172.20.10.11,57756,https://clients1.google.com/complete/search?js...,HTTP/2.0
2,"<< HTTP/2.0 200 OK 148b\n\n date: Sun, 25 ...",[user-agent: Mozilla/5.0 (iPhone; CPU iPhone O...,json: t\n\n nolabels: t\n\n clie...,GET,172.20.10.11,57756,https://clients1.google.com/complete/search?js...,HTTP/2.0
3,"<< HTTP/2.0 200 OK 147b\n\n date: Sun, 25 ...",[user-agent: Mozilla/5.0 (iPhone; CPU iPhone O...,json: t\n\n nolabels: t\n\n clie...,GET,172.20.10.11,57756,https://clients1.google.com/complete/search?js...,HTTP/2.0
4,"<< HTTP/2.0 200 OK 137b\n\n date: Sun, 25 ...",[user-agent: Mozilla/5.0 (iPhone; CPU iPhone O...,json: t\n\n nolabels: t\n\n clie...,GET,172.20.10.11,57756,https://clients1.google.com/complete/search?js...,HTTP/2.0
5,"<< HTTP/2.0 200 OK 137b\n\n date: Sun, 25 ...",[user-agent: Mozilla/5.0 (iPhone; CPU iPhone O...,json: t\n\n nolabels: t\n\n clie...,GET,172.20.10.11,57756,https://clients1.google.com/complete/search?js...,HTTP/2.0
6,"<< HTTP/2.0 200 OK 166b\n\n date: Sun, 25 ...",[user-agent: Mozilla/5.0 (iPhone; CPU iPhone O...,json: t\n\n nolabels: t\n\n clie...,GET,172.20.10.11,57756,https://clients1.google.com/complete/search?js...,HTTP/2.0
7,"<< HTTP/2.0 200 OK 179b\n\n date: Sun, 25 ...",[user-agent: Mozilla/5.0 (iPhone; CPU iPhone O...,json: t\n\n nolabels: t\n\n clie...,GET,172.20.10.11,57756,https://clients1.google.com/complete/search?js...,HTTP/2.0
8,"<< HTTP/2.0 200 OK 126b\n\n date: Sun, 25 ...",[user-agent: Mozilla/5.0 (iPhone; CPU iPhone O...,json: t\n\n nolabels: t\n\n clie...,GET,172.20.10.11,57756,https://clients1.google.com/complete/search?js...,HTTP/2.0
9,"<< HTTP/2.0 200 OK 102b\n\n date: Sun, 25 ...",[user-agent: Mozilla/5.0 (iPhone; CPU iPhone O...,json: t\n\n nolabels: t\n\n clie...,GET,172.20.10.11,57756,https://clients1.google.com/complete/search?js...,HTTP/2.0


# Reordering the dataframe

In [339]:
# Reordering the dataframe to make it easier to read
df_req_res_pair = df_req_res_pair.reindex(columns=['Req.Method', 'Req.Source_IP', 'Req.Source_Port', 'Req.URL', 'Req.HTTP_Version','Req.Headers' ,'Req.Content', 'Res.Start'])

# Printing the dataframe
df_req_res_pair

,Req.Method,Req.Source_IP,Req.Source_Port,Req.URL,Req.HTTP_Version,Req.Headers,Req.Content,Res.Start
0,GET,172.20.10.11,57731,https://p31-quota.icloud.com/quotaservice/exte...,None,"[Host: p31-quota.icloud.com:443, X-MMe-Timezon...",,<< 200 532b\n\n Server: AppleHttpServer/7...
1,GET,172.20.10.11,57756,https://clients1.google.com/complete/search?js...,HTTP/2.0,[user-agent: Mozilla/5.0 (iPhone; CPU iPhone O...,json: t\n\n nolabels: t\n\n clie...,"<< HTTP/2.0 200 OK 152b\n\n date: Sun, 25 ..."
2,GET,172.20.10.11,57756,https://clients1.google.com/complete/search?js...,HTTP/2.0,[user-agent: Mozilla/5.0 (iPhone; CPU iPhone O...,json: t\n\n nolabels: t\n\n clie...,"<< HTTP/2.0 200 OK 148b\n\n date: Sun, 25 ..."
3,GET,172.20.10.11,57756,https://clients1.google.com/complete/search?js...,HTTP/2.0,[user-agent: Mozilla/5.0 (iPhone; CPU iPhone O...,json: t\n\n nolabels: t\n\n clie...,"<< HTTP/2.0 200 OK 147b\n\n date: Sun, 25 ..."
4,GET,172.20.10.11,57756,https://clients1.google.com/complete/search?js...,HTTP/2.0,[user-agent: Mozilla/5.0 (iPhone; CPU iPhone O...,json: t\n\n nolabels: t\n\n clie...,"<< HTTP/2.0 200 OK 137b\n\n date: Sun, 25 ..."
5,GET,172.20.10.11,57756,https://clients1.google.com/complete/search?js...,HTTP/2.0,[user-agent: Mozilla/5.0 (iPhone; CPU iPhone O...,json: t\n\n nolabels: t\n\n clie...,"<< HTTP/2.0 200 OK 137b\n\n date: Sun, 25 ..."
6,GET,172.20.10.11,57756,https://clients1.google.com/complete/search?js...,HTTP/2.0,[user-agent: Mozilla/5.0 (iPhone; CPU iPhone O...,json: t\n\n nolabels: t\n\n clie...,"<< HTTP/2.0 200 OK 166b\n\n date: Sun, 25 ..."
7,GET,172.20.10.11,57756,https://clients1.google.com/complete/search?js...,HTTP/2.0,[user-agent: Mozilla/5.0 (iPhone; CPU iPhone O...,json: t\n\n nolabels: t\n\n clie...,"<< HTTP/2.0 200 OK 179b\n\n date: Sun, 25 ..."
8,GET,172.20.10.11,57756,https://clients1.google.com/complete/search?js...,HTTP/2.0,[user-agent: Mozilla/5.0 (iPhone; CPU iPhone O...,json: t\n\n nolabels: t\n\n clie...,"<< HTTP/2.0 200 OK 126b\n\n date: Sun, 25 ..."
9,GET,172.20.10.11,57756,https://clients1.google.com/complete/search?js...,HTTP/2.0,[user-agent: Mozilla/5.0 (iPhone; CPU iPhone O...,json: t\n\n nolabels: t\n\n clie...,"<< HTTP/2.0 200 OK 102b\n\n date: Sun, 25 ..."


# Creating a dictionary of the headers
- To make the headers easier to work with, we convert the array of strings into a dictionary

In [340]:
def convert_to_dict(header_list):
    header_dict = {}
    for header in header_list:
        key, value = header.split(':', 1)  # Split on the first colon only
        header_dict[key.strip()] = value.strip()  # Remove any leading/trailing whitespace
    return header_dict

# Apply the function to each row in the dataframe for the 'Req.Headers' column
df_req_res_pair['Req.Headers'] = df_req_res_pair['Req.Headers'].apply(convert_to_dict)
df_req_res_pair

,Req.Method,Req.Source_IP,Req.Source_Port,Req.URL,Req.HTTP_Version,Req.Headers,Req.Content,Res.Start
0,GET,172.20.10.11,57731,https://p31-quota.icloud.com/quotaservice/exte...,None,"{'Host': 'p31-quota.icloud.com:443', 'X-MMe-Ti...",,<< 200 532b\n\n Server: AppleHttpServer/7...
1,GET,172.20.10.11,57756,https://clients1.google.com/complete/search?js...,HTTP/2.0,{'user-agent': 'Mozilla/5.0 (iPhone; CPU iPhon...,json: t\n\n nolabels: t\n\n clie...,"<< HTTP/2.0 200 OK 152b\n\n date: Sun, 25 ..."
2,GET,172.20.10.11,57756,https://clients1.google.com/complete/search?js...,HTTP/2.0,{'user-agent': 'Mozilla/5.0 (iPhone; CPU iPhon...,json: t\n\n nolabels: t\n\n clie...,"<< HTTP/2.0 200 OK 148b\n\n date: Sun, 25 ..."
3,GET,172.20.10.11,57756,https://clients1.google.com/complete/search?js...,HTTP/2.0,{'user-agent': 'Mozilla/5.0 (iPhone; CPU iPhon...,json: t\n\n nolabels: t\n\n clie...,"<< HTTP/2.0 200 OK 147b\n\n date: Sun, 25 ..."
4,GET,172.20.10.11,57756,https://clients1.google.com/complete/search?js...,HTTP/2.0,{'user-agent': 'Mozilla/5.0 (iPhone; CPU iPhon...,json: t\n\n nolabels: t\n\n clie...,"<< HTTP/2.0 200 OK 137b\n\n date: Sun, 25 ..."
5,GET,172.20.10.11,57756,https://clients1.google.com/complete/search?js...,HTTP/2.0,{'user-agent': 'Mozilla/5.0 (iPhone; CPU iPhon...,json: t\n\n nolabels: t\n\n clie...,"<< HTTP/2.0 200 OK 137b\n\n date: Sun, 25 ..."
6,GET,172.20.10.11,57756,https://clients1.google.com/complete/search?js...,HTTP/2.0,{'user-agent': 'Mozilla/5.0 (iPhone; CPU iPhon...,json: t\n\n nolabels: t\n\n clie...,"<< HTTP/2.0 200 OK 166b\n\n date: Sun, 25 ..."
7,GET,172.20.10.11,57756,https://clients1.google.com/complete/search?js...,HTTP/2.0,{'user-agent': 'Mozilla/5.0 (iPhone; CPU iPhon...,json: t\n\n nolabels: t\n\n clie...,"<< HTTP/2.0 200 OK 179b\n\n date: Sun, 25 ..."
8,GET,172.20.10.11,57756,https://clients1.google.com/complete/search?js...,HTTP/2.0,{'user-agent': 'Mozilla/5.0 (iPhone; CPU iPhon...,json: t\n\n nolabels: t\n\n clie...,"<< HTTP/2.0 200 OK 126b\n\n date: Sun, 25 ..."
9,GET,172.20.10.11,57756,https://clients1.google.com/complete/search?js...,HTTP/2.0,{'user-agent': 'Mozilla/5.0 (iPhone; CPU iPhon...,json: t\n\n nolabels: t\n\n clie...,"<< HTTP/2.0 200 OK 102b\n\n date: Sun, 25 ..."


In [341]:
# Exports to .csv file
df_req_res_pair.to_csv(exported_csv_filepath, index=False)